In [395]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

In [396]:
state = 'WA'
year = '2023'
inpath = 'F:/Libraries/My Documents/UM/Milestone II/Weather_' + state + '_' + year + '.csv'
outpath = 'F:/Libraries/My Documents/UM/Milestone II/Weather_' + state + '_' + year + '_cleaned.csv'

In [397]:
weather_df = pd.read_csv(inpath, low_memory=False)

In [398]:
# remove that last blank row with the odd character in the station column
weather_df = weather_df.dropna()

In [399]:
# drop the 'attributes' columns and other columns that we don't want
weather_df = weather_df.drop(['TEMP_ATTRIBUTES', 'DEWP_ATTRIBUTES', 'SLP_ATTRIBUTES', 'STP_ATTRIBUTES', 'VISIB_ATTRIBUTES',\
                              'WDSP_ATTRIBUTES', 'MAX_ATTRIBUTES', 'MIN_ATTRIBUTES', 'PRCP_ATTRIBUTES',\
                              'SLP', 'STP', 'DEWP', 'VISIB', 'FRSHTT'], axis=1)

In [400]:
# remove duplicate header rows
weather_df = weather_df[weather_df['STATION'] != 'STATION']

In [401]:
to_numeric_list = ['TEMP', 'WDSP', 'MXSPD', 'GUST', 'MAX', 'MIN', 'PRCP', 'SNDP']
weather_df[to_numeric_list] = weather_df[to_numeric_list].apply(pd.to_numeric)

In [402]:
# clean up dummy 9s values
weather_df.loc[weather_df['TEMP'] >= 999, 'TEMP'] = 50 # not sure this ever happens...
weather_df.loc[weather_df['WDSP'] >= 999, 'WDSP'] = 0
weather_df.loc[weather_df['MXSPD'] >= 999, 'MXSPD'] = 0
weather_df.loc[weather_df['GUST'] >= 999, 'GUST'] = 0
weather_df.loc[weather_df['PRCP'] >= 99, 'PRCP'] = 0
weather_df.loc[weather_df['SNDP'] >= 999, 'SNDP'] = 0

In [403]:
# if MAX temp is below average temp (must be a dummy value), set to average temp of the day
# if MIN temp is above average temp (must be a dummy value), set to average temp of the day
for i in weather_df.index:
    daily_avg_temp = weather_df['TEMP'][i]
    if weather_df['MAX'][i] < daily_avg_temp:
        weather_df['MAX'][i] = daily_avg_temp
    if weather_df['MAX'][i] >= 999:
        weather_df['MAX'][i] = daily_avg_temp
    if weather_df['MIN'][i] > daily_avg_temp:
        weather_df['MIN'][i] = daily_avg_temp

In [404]:
# create new temp extremity measure

# default to 0
weather_df['TEMPEXT_BASE40'] = abs(weather_df['TEMP']-40)
weather_df['TEMPEXT_BASE45'] = abs(weather_df['TEMP']-45)
weather_df['TEMPEXT_BASE50'] = abs(weather_df['TEMP']-50)
    

In [405]:
weather_df.to_csv(outpath, index=False)